# HW4 Part B: Expectation Maximization for Digits Data Set

Blanca Miller
<br>
CS 791
<br>
03/01/2018

__Objective:__ Determine the number of Gaussians needed to represent a data set. Using the general technique for finding maximum 

The Expecatation Maximization (EM) algorithm is an iterative procedure composed of two main processes: the expectation step and the maximization step. The expectation step calculates the expected values, that is the probability that the data point x was drawn from a jth distribution, given the jth distribution's mean, mu, divided by the sum of probabilities that x belonged to each mu. Then, the maximization step recomputes the mean and covariance given a current position. This process is repeated until the local maximum likelihood parameters. 


## STEPS

Setup:
1. Import Libraries 
2. Import data sets: training and testing
3. Convert data frames into numpy arrays
4. Parse the data sets into two matrices:
    - X: design matrix
    - y: targets/labels matrix
5. Standardize the training & testing design matrices

EM Algorithm
1. Generate random samples
2. Iterate through parameters for t=0, 1, 2 ... until convergence:
    - Expectation-step: Compute the conditional expectation under the given parameter
    - Maximization-step: Choose the parameter theta at t+1 as the maximizer, then maximize the Q function for parameter theta with theta at t 
3. Repeat

## DATA SET
- 7291 training observations
- 2007 testing observations
- 16 x 16 grayscale images of digits
- Each row contains a digit id (0-9) followed by its 256 grayscale values

## Import Libraries

In [1]:
import sklearn 
import numpy as np
import pandas as pd
import scipy
import itertools
import matplotlib.pyplot as plt

from sklearn import mixture
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.datasets.samples_generator import make_blobs
from scipy import linalg
from scipy.stats import multivariate_normal as mvn

## Import Data Sets

In [2]:
# Initialize an object for the training and testing data sets
train = pd.read_csv('digits_data.train', delimiter=' ', header=None)
test = pd.read_csv('digits_data.test', delimiter=' ', header=None)

# print size of data sets
print("Training Set: {}".format(train.shape))
print("Testing Set: {}".format(test.shape))

Training Set: (7291, 258)
Testing Set: (2007, 257)


In [3]:
# print observations for training set
train.head()

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,256,257
0,6.0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-0.631,0.862,...,0.823,1.000,0.482,-0.474,-0.991,-1.000,-1.000,-1.000,-1.0,NaN
1,5.0,-1.0,-1.0,-1.0,-0.813,-0.671,-0.809,-0.887,-0.671,-0.853,...,-0.671,-0.033,0.761,0.762,0.126,-0.095,-0.671,-0.828,-1.0,NaN
2,4.0,-1.0,-1.0,-1.0,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,-0.109,1.000,-0.179,-1.000,-1.000,-1.000,-1.0,NaN
3,7.0,-1.0,-1.0,-1.0,-1.000,-1.000,-0.273,0.684,0.960,0.450,...,1.000,0.536,-0.987,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0,NaN
4,3.0,-1.0,-1.0,-1.0,-1.000,-1.000,-0.928,-0.204,0.751,0.466,...,0.639,1.000,1.000,0.791,0.439,-0.199,-0.883,-1.000,-1.0,NaN


In [4]:
# print observations for testing data set
test.head()

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,256
0,9,-1.0,-1.0,-1.0,-1.000,-1.0,-0.948,-0.561,0.148,0.384,...,-1.000,-0.908,0.430,0.622,-0.973,-1.000,-1.0,-1.0,-1.0,-1.0
1,6,-1.0,-1.0,-1.0,-1.000,-1.0,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.0
2,3,-1.0,-1.0,-1.0,-0.593,0.7,1.000,1.000,1.000,1.000,...,1.000,0.717,0.333,0.162,-0.393,-1.000,-1.0,-1.0,-1.0,-1.0
3,6,-1.0,-1.0,-1.0,-1.000,-1.0,-1.000,-1.000,-1.000,-1.000,...,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.0
4,6,-1.0,-1.0,-1.0,-1.000,-1.0,-1.000,-1.000,-0.858,-0.106,...,0.901,0.901,0.901,0.290,-0.369,-0.867,-1.0,-1.0,-1.0,-1.0


## Convert the Data Frames into Numpy Arrays

In [5]:
train_set = train.as_matrix()
test_set = test.as_matrix()

## Parse the Data (X) from the Responses/Labels (y)

In [6]:
# for all rows, start at the 1st column and go until the end of the column
X_train = train_set[:,1:257] # got to 256 to remove NaN column that numpy inserted

# for alls rows, get only the 0th element
y_train = train_set[:,0]

# for all rows, start at the 1st column and go until the end of the column
X_test = test_set[:,1:]

# for all rows, get only the 0th element
y_test = test_set[:,0]

# Number of training samples (rows)
n_samples = X_train.shape[0]

# Number of features (columns)
n_features = X_train.shape[1]

print("Training Data: {}".format(X_train.shape))
print("Training Labels: {}\n".format(y_train.shape))
print("Testing Data: {}".format(X_test.shape))
print("Testing Labels: {}\n".format(y_test.shape))
print("Number of Training Samples: {}".format(n_samples))
print("Number of Data Features: {}".format(n_features))

Training Data: (7291, 256)
Training Labels: (7291,)

Testing Data: (2007, 256)
Testing Labels: (2007,)

Number of Training Samples: 7291
Number of Data Features: 256


## Standardize X for Similar Input & Weight Magnitudes

In [7]:
# Set axis to 1 to standardize per sample/vector, rather than standardize each feature
X_train = preprocessing.scale(X_train, axis=1)
X_test = preprocessing.scale(X_test, axis=1)
print("Standardized Training Data: \n{}".format(X_train))
print("Standardized Testing Data: \n{}".format(X_test))

Standardized Training Data: 
[[-0.80693359 -0.80693359 -0.80693359 ..., -0.80693359 -0.80693359
  -0.80693359]
 [-1.0229121  -1.0229121  -1.0229121  ..., -0.64403944 -0.82483886
  -1.0229121 ]
 [-0.62434198 -0.62434198 -0.62434198 ..., -0.62434198 -0.62434198
  -0.62434198]
 ..., 
 [-0.88870472 -0.88870472 -0.88870472 ..., -0.88870472 -0.88870472
  -0.88870472]
 [-1.34819665 -1.34819665 -1.34819665 ..., -1.34819665 -1.34819665
  -1.34819665]
 [-0.66492726 -0.66492726 -0.66492726 ..., -0.66492726 -0.66492726
  -0.66492726]]
Standardized Testing Data: 
[[-0.69066866 -0.69066866 -0.69066866 ..., -0.69066866 -0.69066866
  -0.69066866]
 [-0.69349172 -0.69349172 -0.69349172 ..., -0.69349172 -0.69349172
  -0.69349172]
 [-0.80198143 -0.80198143 -0.80198143 ..., -0.80198143 -0.80198143
  -0.80198143]
 ..., 
 [-0.74336125 -0.74336125 -0.74336125 ..., -0.74336125 -0.74336125
  -0.74336125]
 [-1.11691895 -1.11691895 -1.11691895 ..., -1.11691895 -1.11691895
  -1.11691895]
 [-0.52395781 -0.52395781 

## EM Function

In [8]:
def em_gmm_orig(xs, pis, mus, sigmas, tol=0.01, max_iter=100):

    n, p = xs.shape
    k = len(pis)

    ll_old = 0
    for i in range(max_iter):
        exp_A = []
        exp_B = []
        ll_new = 0

        # E-step
        # ===========================================================
        ws = np.zeros((k, n))
        for j in range(len(mus)):
            for i in range(n):
                print("ws: {}".format(ws.shape))
                print("pis: {}".format(pis.shape)) 
                print("sigmas: {}".format(sigmas.shape))
                ws[j, i] = pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
        ws /= ws.sum(0)
        
        # M-step
        # ===========================================================
        pis = np.zeros(k)
        for j in range(len(mus)):
            for i in range(n):
                pis[j] += ws[j, i]
        pis /= n

        mus = np.zeros((k, p))
        for j in range(k):
            for i in range(n):
                mus[j] += ws[j, i] * xs[i]
            mus[j] /= ws[j, :].sum()

        sigmas = np.zeros((k, p, p))
        for j in range(k):
            for i in range(n):
                ys = np.reshape(xs[i]- mus[j], (2,1))
                sigmas[j] += ws[j, i] * np.dot(ys, ys.T)
            sigmas[j] /= ws[j,:].sum()

        # update complete log likelihoood
        ll_new = 0.0
        for i in range(n):
            s = 0
            for j in range(k):
                s += pis[j] * mvn(mus[j], sigmas[j]).pdf(xs[i])
            ll_new += np.log(s)

        if np.abs(ll_new - ll_old) < tol:
            break
        ll_old = ll_new

    return ll_new, pis, mus, sigmas

## EM

In [19]:
# initialize random values for parameters
np.random.seed(0)

parameters = np.random.random(2)
parameters /= parameters.sum()

mus = np.random.random((2,2))
sigmas = np.array([np.eye(2)] * 2) * X_train.std()


print("Random Parameters:\n {}\n".format(parameters))
print("Mean Distributions:\n {}\n".format(mus))
print("Covariances:\n {}\n".format(sigmas))

print("Size of Random Parameters: {}".format(parameters.shape))
print("Size of Random Mean Distributions: {}".format(mus.shape))
print("Covariances: {}".format(sigmas.shape))
print("Size of Random Values Matrix: {}".format(X_train.shape))

Random Parameters:
 [ 0.43418691  0.56581309]

Mean Distributions:
 [[ 0.60276338  0.54488318]
 [ 0.4236548   0.64589411]]

Covariances:
 [[[ 1.  0.]
  [ 0.  1.]]

 [[ 1.  0.]
  [ 0.  1.]]]

Size of Random Parameters: (2,)
Size of Random Mean Distributions: (2, 2)
Covariances: (2, 2, 2)
Size of Random Values Matrix: (7291, 256)


In [ ]:
ll1, params1, mus1, sigmas1 = em_gmm_orig(X_train, parameters, mus, sigmas)